In [20]:
!pip install prophet

In [21]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
df = pd.read_csv("cafethai_prices.csv",parse_dates=['date'], date_format='%d/%m/%Y')


daily_sales = df.groupby(['date', 'coffee_name']).size().reset_index(name='cups_sold')

df
daily_sales

,date,coffee_name,cups_sold
0,2023-03-01,Americano,19
1,2023-03-01,Americano with Milk,27
2,2023-03-01,Cappuccino,24
3,2023-03-01,Cocoa,7
4,2023-03-01,Cortado,14
...,...,...,...
6166,2025-04-30,Cocoa,3
6167,2025-04-30,Cortado,6
6168,2025-04-30,Espresso,2
6169,2025-04-30,Hot Chocolate,7


In [23]:
from prophet import Prophet

drink_types = daily_sales['coffee_name'].unique()
all_forecasts = []

for drink in drink_types:
    drink_data = (
        daily_sales[daily_sales['coffee_name'] == drink][['date', 'cups_sold']]
        .copy()
        .rename(columns={'date': 'ds', 'cups_sold': 'y'})
    )

    model = Prophet(daily_seasonality=True, yearly_seasonality=True)
    model.fit(drink_data)

    future = model.make_future_dataframe(periods=60)
    forecast = model.predict(future)

    future_forecast = (
        forecast.tail(60)[['ds', 'yhat']]
        .assign(coffee_name=drink)
    )

    all_forecasts.append(future_forecast)

result = (
    pd.concat(all_forecasts, ignore_index=True)
    .rename(columns={'ds': 'date', 'yhat': 'predicted_cups'})
)


DEBUG:cmdstanpy:input tempfile: /tmp/tmpp64bkwd1/6fuhu6q2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp64bkwd1/0yfjbjin.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=95497', 'data', 'file=/tmp/tmpp64bkwd1/6fuhu6q2.json', 'init=/tmp/tmpp64bkwd1/0yfjbjin.json', 'output', 'file=/tmp/tmpp64bkwd1/prophet_model2yrb41lu/prophet_model-20250921151036.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:10:36 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:10:36 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp64bkwd1/y25sdxir.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp64bkwd1/lcw8y8za.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

In [24]:
table_result = result.pivot(index='date', columns='coffee_name', values='predicted_cups')
display(table_result)

coffee_name,Americano,Americano with Milk,Cappuccino,Cocoa,Cortado,Espresso,Hot Chocolate,Latte
date,,,,,,,,
2025-05-01,9.506918,14.468708,8.800736,4.499558,4.037747,2.437751,5.613302,12.140996
2025-05-02,7.936232,12.981205,8.127557,4.347671,3.797445,2.030500,5.435393,10.997782
2025-05-03,1.759430,3.170118,0.938922,1.452298,0.327046,0.380951,1.783599,1.116464
2025-05-04,1.788706,3.507020,0.681930,1.333529,0.127796,0.745275,1.563881,0.855342
2025-05-05,8.662543,12.602627,6.794763,3.978535,4.154370,1.975037,5.127058,10.582619
2025-05-06,7.997170,12.214978,6.761022,4.007834,3.526978,2.291724,4.454973,9.887765
2025-05-07,8.854686,12.720374,6.806261,4.164641,3.812778,2.155455,5.114559,9.211931
2025-05-08,7.932463,11.853628,7.306653,3.910213,3.195197,2.456721,4.420550,9.816345
2025-05-09,6.401866,10.387919,6.628811,3.763200,3.035218,2.051479,4.227261,8.751323


In [40]:
import plotly.express as px

fig = px.line(result, x="date", y="predicted_cups", color="coffee_name")
fig.show()